In [ ]:
import pandas as pd
from openai import OpenAI
import requests
import time

In [ ]:
data = pd.read_csv('transformed_data_men_21_to_24.csv')
df=data.copy()
GOOGLE_API = ''
ENGINE_ID = ''
teams = list(df['team'].unique())
google_dict={}
url = 'https://www.googleapis.com/customsearch/v1'

#iterate through each team to query google for recent data
for team in teams:
    search_query = f"What is the strength of the 2024 {team} Mens basketball team?"
    
    #restricts query results to this year/season
    params = {
    'q': search_query,
    'key': GOOGLE_API,
    'cx': ENGINE_ID,
    'dateRestrict': '2023-06-01:2024-02-23'
    }
    response = requests.get(url, params=params)
    results = response.json()
    summary = []
    for result in results['items']:
        try:
            for item in result['pagemap']['metatags']:
                summary.append(item['og:description'])
        except:
            summary.append(result['snippet'])
    print(team)
    google_dict[team] = summary
    #include sleep function due to API errors
    time.sleep(5)
    

google_df = pd.DataFrame(google_dict)

google_df.to_csv('ncaa_google_search_data_new.csv')

In [ ]:
data = pd.read_csv('transformed_data_men_21_to_24.csv')
google_data = pd.read_csv('ncaa_google_search_data_new.csv')
#google_data = google_data.transpose()


#generates chatgpt responses
def chat_results(teams, google_results):
    team_dict = {}
    total_calls = 0
    client=OpenAI(api_key='')
    for team in teams:
        team_search = list(google_results[team])
        team_search = str(team_search)
        prompt = f"Provide a strength of team score on a scale of 1-100 for the 2023-2024 {team} Men's basketball team based on their performance and the quality of the teams they have played, and the following Google search reslts. Please provide only the score value. Google Result: "+ team_search
        while True:
            completion = client.chat.completions.create(
                model="gpt-4-0125-preview",
                messages=[
                    {"role": "system", "content": "You are a sports analyst skilled in evaluating NCAA teams."},
                    {"role": "user", "content": prompt}
                ]
            )
            total_calls+=1
            sent_score = completion.choices[0].message.content
            print(prompt)
            print('Score: ', sent_score)
            print('Total Calls: ', total_calls)
            if len(sent_score) < 4:
                sent_score = int(sent_score)
                break
            if total_calls > 900:
                print('Too Many Calls!! Fix Something!!')
                print('Total Calls > 900')
                break
        team_dict[team] = sent_score
    return team_dict


#provides a sentiment analysis score for a team based on ChatGPT
def sent_data(df, google_results):
    teams = list(df['team'].unique())

    #perfrom the ChatGPT function three times
    team_dict1 =  chat_results(teams, google_results)   
    df['sent_score1'] = df['team'].map(team_dict1)
    team_dict2 =  chat_results(teams, google_results)   
    df['sent_score2'] = df['team'].map(team_dict2)
    team_dict3 =  chat_results(teams, google_results)   
    df['sent_score3'] = df['team'].map(team_dict3)

    return df

merged_data = sent_data(data, google_data)

merged_data.to_csv('transformed_data_men_21_to_24_chatgpt')